webcam

In [ ]:
import cv2
from deepface import DeepFace
import serial
import time

# Set up serial communication with Arduino
arduino = serial.Serial('COM7', 9600)  # Change 'COM3' to your Arduino port
time.sleep(2)  # Wait for the connection to establish

# Load the face detector
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start video capture
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Analyze frame for emotion
    result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

    # Draw rectangles and emotions
    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Get the dominant emotion
        emotion = result[0]['dominant_emotion']
        cv2.putText(
            frame,
            emotion,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            cv2.LINE_4
        )

        # Check if the emotion is 'sad'
        if emotion == 'sad':
            arduino.write(b'0')  # Send signal to Arduino to activate mist maker
        else:
            arduino.write(b'1')  # Send signal to Arduino to deactivate mist maker

    # Show the frame
    cv2.imshow('Demo video', frame)

    # Quit on pressing 'q'
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
arduino.close()  # Close the serial connection

webcam with music

In [ ]:
import cv2
from deepface import DeepFace
import numpy as np
import pygame  # For music playback
import serial  # For Arduino communication
import time

# Set up serial communication with Arduino
arduino = serial.Serial('COM7', 9600)  # Change 'COM7' to match your Arduino port
time.sleep(2)  # Wait for the connection to establish

# Initialize pygame mixer
pygame.mixer.init()

# Load the face detector
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start video capture
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Function for gamma correction
def apply_gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

# Function to enhance exposure and contrast
def enhance_frame(frame):
    frame = apply_gamma_correction(frame, gamma=1.5)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    enhanced_frame = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return enhanced_frame

# Play music based on emotion
current_emotion = None
music_tracks = {
    "happy": r"G:\ad\music\happy.mp3",
    "sad": r"G:\ad\music\sad.mp3",
}

# Function to play music based on emotion
def play_music(emotion):
    global current_emotion

    if emotion in music_tracks:
        if current_emotion != emotion:
            current_emotion = emotion
            pygame.mixer.music.stop()
            pygame.mixer.music.load(music_tracks[emotion])
            pygame.mixer.music.play()
            # Send emotion to Arduino
            arduino.write((emotion + "\n").encode())
    else:
        pygame.mixer.music.stop()
        arduino.write(b"none\n")

def analyze_emotion(frame):
    try:
        result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        return result[0]['dominant_emotion']
    except Exception as e:
        print(f"Emotion analysis error: {e}")
        return None

# Main loop
frame_count = 0
emotion = None  # Initialize here to prevent undefined error

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = enhance_frame(frame)

    # Analyze emotion every 5 frames
    if frame_count % 5 == 0:
        detected_emotion = analyze_emotion(frame)
        if detected_emotion:
            emotion = detected_emotion
            play_music(emotion)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(
            frame,
            emotion if emotion else "No Emotion",
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            cv2.LINE_4
        )

    cv2.imshow('Emotion-based Music Player', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()
arduino.close()  # Close the serial connection

To run your **emotion-based music player with Arduino integration**, you'll need to install several **Python libraries**, ensure **hardware setup**, and have some **media files** and **Arduino code** ready. Here's a full list of what you need to install and prepare:

---

### ✅ PYTHON LIBRARIES TO INSTALL

You can install them via `pip`:

```bash
pip install opencv-python deepface numpy pygame pyserial
```

**Explanation of each:**

| Library       | Purpose                                           |
|---------------|---------------------------------------------------|
| `opencv-python` | Video capture and face detection               |
| `deepface`      | Emotion detection using deep learning          |
| `numpy`         | Image manipulation (gamma correction, etc.)    |
| `pygame`        | Music playback                                 |
| `pyserial`      | Communication with Arduino via serial          |

**Note:** `deepface` may require additional dependencies (like `tensorflow`, `keras`, `mtcnn`). They should install automatically with DeepFace, but if not, you can install manually:

```bash
pip install tensorflow keras mtcnn
```

---

### ✅ SYSTEM REQUIREMENTS

1. **Python 3.7–3.11** (recommended)
2. **Camera** (webcam)
3. **Speakers** (for audio playback)

---

### ✅ HARDWARE SETUP

- **Arduino board** (Uno, Nano, etc.)
- Connect Arduino to your PC via USB
- Make sure your Arduino is programmed to respond to serial inputs like `"happy\n"` or `"sad\n"` — this can control LEDs or motors, etc.

---

### ✅ MUSIC FILES

Put the correct music files in the specified paths:
```python
music_tracks = {
    "happy": r"G:\ad\music\happy.mp3",
    "sad": r"G:\ad\music\sad.mp3",
}
```
Make sure these files exist, or change the paths accordingly.

---

### ✅ ARDUINO SIDE CODE (Example)

Upload this via Arduino IDE:

```cpp
void setup() {
  Serial.begin(9600);
  pinMode(13, OUTPUT);  // Example pin
}

void loop() {
  if (Serial.available()) {
    String emotion = Serial.readStringUntil('\n');
    if (emotion == "happy") {
      digitalWrite(13, HIGH); // Happy mode
    } else if (emotion == "sad") {
      digitalWrite(13, LOW);  // Sad mode
    } else {
      digitalWrite(13, LOW);  // Default
    }
  }
}
```

---

### ✅ OPTIONAL: ENVIRONMENT SETUP

- **Virtual environment** (recommended to avoid version conflicts):
```bash
python -m venv emo_env
emo_env\Scripts\activate
```

---

### ✅ SUMMARY CHECKLIST

| ✅ Item                      | Description                          |
|-----------------------------|--------------------------------------|
| Python + pip                | Installed on your system             |
| Required libraries          | Installed via `pip`                  |
| Webcam                      | Working and accessible               |
| Arduino                     | Connected and code uploaded          |
| Music files                 | In correct paths                     |
| Arduino serial port         | Matches `'COM7'` or change if needed |
| Proper permissions          | To access camera & audio             |

---

Want me to create a simple interface to test it or generate Arduino code based on multiple emotions (like angry, neutral, etc.)?

In [ ]:
// Arduino Code: Emotion-based Mist Maker Control

#define RELAY_HAPPY 7  // Relay 1 - Mist Maker 1
#define RELAY_SAD 8    // Relay 2 - Mist Maker 2

String inputString = "";

void setup() {
  pinMode(RELAY_HAPPY, OUTPUT);
  pinMode(RELAY_SAD, OUTPUT);
  
  // Start with both mist makers OFF
  digitalWrite(RELAY_HAPPY, LOW);
  digitalWrite(RELAY_SAD, LOW);

  Serial.begin(9600);
}

void loop() {
  while (Serial.available()) {
    char c = Serial.read();
    if (c == '\n') {
      inputString.trim();

      if (inputString == "happy") {
        digitalWrite(RELAY_HAPPY, HIGH); // Turn ON happy mist
        digitalWrite(RELAY_SAD, LOW);    // Turn OFF sad mist
      } 
      else if (inputString == "sad") {
        digitalWrite(RELAY_HAPPY, LOW);
        digitalWrite(RELAY_SAD, HIGH);
      } 
      else {
        digitalWrite(RELAY_HAPPY, LOW);
        digitalWrite(RELAY_SAD, LOW);
      }

      inputString = "";  // Clear buffer
    } else {
      inputString += c;
    }
  }
}


In [ ]:
import cv2
from deepface import DeepFace
import numpy as np
import pygame
import serial
import time

# Connect to Arduino (change 'COM7' as needed)
arduino = serial.Serial('COM7', 9600)
time.sleep(2)

# Initialize pygame for music
pygame.mixer.init()

# Load face detector
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(1)

def apply_gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

def enhance_frame(frame):
    frame = apply_gamma_correction(frame)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

music_tracks = {
    "happy": r"D:\ad\music\happy.mp3",
    "sad": r"D:\ad\music\sad.mp3",
}

current_emotion = None

def play_music_and_trigger_relay(emotion):
    global current_emotion
    if emotion in music_tracks:
        if current_emotion != emotion:
            current_emotion = emotion
            pygame.mixer.music.stop()
            pygame.mixer.music.load(music_tracks[emotion])
            pygame.mixer.music.play()
            arduino.write((emotion + "\n").encode())
    else:
        pygame.mixer.music.stop()
        arduino.write(b"none\n")

def analyze_emotion(frame):
    try:
        result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        return result[0]['dominant_emotion']
    except Exception as e:
        print(f"Emotion analysis error: {e}")
        return None

frame_count = 0
emotion = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = enhance_frame(frame)

    if frame_count % 50 == 0:
        detected_emotion = analyze_emotion(frame)
        if detected_emotion:
            if detected_emotion in ['happy', 'sad']:
                emotion = detected_emotion
                play_music_and_trigger_relay(emotion)
            else:
                arduino.write(b"none\n")

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(
            frame,
            emotion if emotion else "No Emotion",
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2
        )

    cv2.imshow('Emotion-based Mist Maker Control', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()
arduino.close()



pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


<h1>with custum model</h1>

In [ ]:
import cv2
import numpy as np
import pygame  # For music playback
import serial
import time
from keras.models import load_model
from keras.preprocessing.image import img_to_array

# Connect to Arduino (change 'COM7' as needed)
arduino = serial.Serial('COM7', 9600)
time.sleep(2)

# Initialize pygame mixer
pygame.mixer.init()

# Load your trained model
model = load_model(r"D:\ad\emotion_model_datset.h5")

# Define class labels based on your training
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Load the face detector
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start video capture
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Function for gamma correction
def apply_gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

# Function to enhance exposure and contrast
def enhance_frame(frame):
    frame = apply_gamma_correction(frame, gamma=1.5)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    enhanced_frame = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return enhanced_frame

# Play music based on emotion
current_emotion = None
music_tracks = {
    "happy": r"D:\ad\music\happy.mp3",
    "sad": r"D:\ad\music\sad.mp3"
}

# Function to play music based on emotion
def play_music(emotion):
    global current_emotion
    if emotion in music_tracks:
        if current_emotion != emotion:
            current_emotion = emotion
            pygame.mixer.music.stop()
            pygame.mixer.music.load(music_tracks[emotion])
            pygame.mixer.music.play()
    else:
        pygame.mixer.music.stop()

# Analyze emotion using your custom model
def analyze_emotion_custom(face_img):
    try:
        face_img = cv2.resize(face_img, (48, 48))
        face_img = face_img.astype("float") / 255.0
        face_img = img_to_array(face_img)
        face_img = np.expand_dims(face_img, axis=0)
        preds = model.predict(face_img, verbose=0)[0]
        return emotion_labels[np.argmax(preds)]
    except Exception as e:
        print(f"Emotion analysis error: {e}")
        return None

# Main loop
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = enhance_frame(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

    detected_emotion = None

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        detected_emotion = analyze_emotion_custom(roi_gray)

        if frame_count % 20 == 0 and detected_emotion:
            play_music(detected_emotion)

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(
            frame,
            detected_emotion if detected_emotion else "No Emotion",
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            cv2.LINE_4
        )

    cv2.imshow('Emotion-based Music Player', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()
arduino.close()

In [4]:
pip install pyserial


Note: you may need to restart the kernel to use updated packages.
